# RusT Petalo

In [1]:
import struct
from collections import namedtuple
import os.path
import subprocess

In [5]:
import numpy as np
from operator import mul
from functools import reduce

import matplotlib.pyplot as plt

In [6]:
from sys import argv
import argparse
from itertools import starmap

In [8]:
from sklearn.cluster import KMeans

# utils

In [9]:
def wrap_1d_into_3d(data, shape, row_major=False):
    assert len(data) == reduce(mul, shape)
    image = np.zeros(shape)
    nx, ny, nz = shape
    for index, x in enumerate(data):
        i = int(index % nx)
        j = int(index / nx) % ny
        k = int(index / (nx * ny))
        if row_major: image[k,j,i] = x
        else        : image[i,j,k] = x
    return image

In [10]:
def read_raw(filename):
    buffer = open(filename, 'rb').read()
    metadata_length = 18
    metadata = buffer[: metadata_length  ]
    data     = buffer[  metadata_length :]
    pixels = struct.unpack_from('>HHH', metadata[:6])
    mm     = struct.unpack_from('>fff', metadata[6:])
    data   = struct.unpack_from('>'+'f' * (len(data) // 4), data)
    return (pixels, mm), data

In [11]:
def show_image(image, *, axis, slice_, extent, shape, ax=plt):
    xe, ye, ze = (e/2 for e in extent)

    if axis == 'x': e, n = xe, shape[0]
    if axis == 'y': e, n = ye, shape[1]
    if axis == 'z': e, n = ze, shape[2]

    s = int(n * (slice_ / (2*e) - 0.5))

    if axis == 'x': ax.imshow(image[s, :, :]   , extent = [-ze,ze, -ye,ye], origin = 'lower')
    if axis == 'y': ax.imshow(image[:, s, :]   , extent = [-ze,ze, -xe,xe], origin = 'lower')
    if axis == 'z': ax.imshow(image[:, :, s].T , extent = [-xe,xe, -ye,ye], origin = 'lower')


In [12]:
def show_image_from_file(filename, *, axis, slice_, ax=plt):
    (shape, full_length), data = read_raw(filename)
    image = wrap_1d_into_3d(data, shape)
    show_image(image, axis=axis, slice_=slice_, extent=full_length, shape=shape, ax=ax)


In [13]:
def load_image_from_file(filename):
    (shape, full_extent), data = read_raw(filename)
    image = wrap_1d_into_3d(data, shape)
    return image, full_extent


In [14]:
def display_reconstructed_images(nr, nc, args):
    generate_data_if_missing(args)
    fig, ax = plt.subplots(nr,nc, figsize=(nc*5, nr*5))
    fig.suptitle("C" if args.c else "Rust")

    for (r,c) in ((r,c) for r in range(nr) for c in range(nc)):
        filename = args_to_filename(args, c+nc*r)
        the_ax = ax[c] if min(nr,nc) == 1 else ax[r,c]
        show_image_from_file(filename, axis=args.axis, slice_=args.slice_, ax=the_ax)


In [18]:
def voxel_centres(image, full_extent):
    def coordinates_of_voxel_centres(n, dx):
        last_centre = (n-1) / n * dx / 2
        return np.linspace(-last_centre, last_centre, num=n)
    x,y,z = starmap(coordinates_of_voxel_centres, zip(image.shape, full_extent))
    coords = np.swapaxes(np.array(np.meshgrid(z,y,x)), 0,3)
    return np.flip(coords, axis=3)


In [24]:
def zone(x):
    if x > 150: return 2
    if x >  50: return 1
    return 0

def by_zone(xyz): return tuple(map(zone, xyz))


def slice_to_position(slice_number, n_pixels, full_extent):
    pixel_width = full_extent / n_pixels
    return pixel_width * (slice_number + 0.5 - n_pixels / 2)

def position_to_slice(position, n_pixels, full_extent):
    return round((n_pixels * position) / full_extent + 0.5 * (n_pixels - 1))


In [23]:
def analyse_one_point(x,y,z):

    ylo = position_to_slice(y-delta, ny, ey)
    yhi = position_to_slice(y+delta, ny, ey) + 1

    zlo = position_to_slice(z-delta, nz, ez)
    zhi = position_to_slice(z+delta, nz, ez) + 1

    around_peak = image[:, ylo:yhi, zlo:zhi]
    xpk, ypk, zpk = np.unravel_index(np.argmax(around_peak), around_peak.shape)
    peak_value = around_peak[xpk, ypk, zpk]

    fig, axs = plt.subplots(1,3, tight_layout=True)

    along_x = around_peak[ : , ypk, zpk]
    along_y = around_peak[xpk,  : , zpk]
    along_z = around_peak[xpk, ypk,  : ]

    print(f'{x:6.1f} {y:6.1f} {z:6.1f}     ', end='')

    widths = []
    for i, (a, d) in enumerate(zip((along_x, along_y, along_z), (dx, dy, dz))):
        hm = peak_value /  2
        tm = peak_value / 10
        lh, rh = full_width_at(hm, a, d)
        lt, rt = full_width_at(tm, a, d)
        ax = axs[i]
        ax.plot(a, '-o')
        ax.plot((lh, rh), (hm, hm))
        ax.plot((lt, rt), (tm, tm))
        fig.suptitle(f'{x:6.1f} {y:6.1f} {z:6.1f}')
        ax.set_title(f'FWHM = {rh-lh:4.1f} mm   FWTM = {rt-lt:4.1f} mm')
        widths.append((rh-lh, rt-lt))

    (hx,tx), (hy,ty), (hz,tz) = widths
    print(f'   {hx:4.1f} {hy:4.1f} {hz:4.1f}     {tx:4.1f} {ty:4.1f} {tz:4.1f}')


def n_pixels_to(height, array):
    for i, (this, prev) in enumerate(zip(array[1:], array), 1):
        if this > height:
            return i - 0.5 + (height - prev) / (this - prev)

def full_width_at(height, array, pixel_size):
    l = n_pixels_to(height, array)
    r = n_pixels_to(height, array[::-1])
    return (l - 0.5) * pixel_size, (len(array) - r - 0.5) * pixel_size


## Code

In [15]:
infile = "/Users/jj/JuliaProjects/ANema/data/mlem-zstd-n3-w-20mm-all-average-recall/img_04.raw"

In [16]:
image, full_extent = load_image_from_file(infile)

In [19]:
positions_of_active_voxels = voxel_centres(image, full_extent)[image > 1]

In [20]:
positions_of_active_voxels

array([[ -50., -185.,  125.],
       [ -38., -250.,  125.],
       [ -36., -249.,  125.],
       ...,
       [  20.,   94.,    4.],
       [  21.,  100.,    4.],
       [  50., -137.,  125.]])

In [21]:
kmeans = KMeans(n_clusters=6, random_state=0).fit(positions_of_active_voxels)

In [22]:
kmeans

KMeans(n_clusters=6, random_state=0)

In [25]:
cluster_centres = sorted(kmeans.cluster_centers_, key=by_zone)


In [26]:
cluster_centres

[array([1.02841042e-01, 9.79196995e+00, 7.04390702e-04]),
 array([ -18.5 , -205.25,  125.  ]),
 array([ 4.54164683, 97.72727273, -0.18514993]),
 array([-4.41996047, 99.55237154,  0.14772727]),
 array([4.74796011e+00, 1.98022212e+02, 1.26926564e-02]),
 array([-5.05733945e+00,  1.97200917e+02,  8.30275229e-02])]

In [27]:
delta = 25
ex, ey, ez = full_extent
nx, ny, nz = image.shape
dx, dy, dz = (e/n for e,n in zip(full_extent, image.shape))


In [29]:
print(ex, ey, ex)
print(nx, ny, nz)
print(dx,dy,dz)

101.0 501.0 101.0
101 501 251
1.0 1.0 1.0
